In [1]:
%ls
%run DataAndEnvironment.ipynb

Cameras.ipynb
data@
DataAndEnvironment.ipynb
data.py
model.py
nerp.py
nerv.py
__pycache__/
README.md
test_camera_0_features_-0.0108_0.3133_-0.7449_0.0179_0.2330_-0.9397.png
test_camera_1_features_-0.2786_-0.6991_0.8099_0.4372_-0.0442_-0.0797.png
test_camera_2_features_0.9018_-0.8738_-0.6735_0.9631_0.6722_0.7186.png
test_camera_3_features_0.6825_0.3112_0.9095_0.5384_-0.2164_-0.5489.png
test_camera_4_features_-0.3895_0.5603_0.8177_0.4621_-0.3504_-0.9236.png
test_camera_5_features_-0.3143_-0.8350_0.9672_0.2261_0.4855_-0.6276.png
test_camera_6_features_0.2507_0.2731_0.1764_-0.3692_-0.8947_0.8825.png
test_camera_7_features_0.0704_0.8853_0.8612_0.9380_0.9240_-0.7961.png
test_data_full_01/
test_data_window_-200_1500/
test_data_window_-500_3071/
VolRen.ipynb
1593
['data/ChestXRLungSegmentation/Imagenglab/processed/train/images/001.nii.gz']
23389
['data/ChestXRLungSegmentation/VinDr/v1/processed/train/images/000434271f63a053c4128a0ba6352c7f.png']
1593
['data/ChestXRLungSegmentation/Imagenglab/p

In [2]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
# cameras = RandomCameras(batch_size=hparams.batch_size, random=True).to(device)

# render_size describes the size of both sides of the 
# rendered images in pixels. We set this to the same size
# as the target images. I.e. we render at the same
# size as the ground truth images.
render_size = hparams.shape

# Our rendered scene is centered around (0,0,0) 
# and is enclosed inside a bounding box
# whose side is roughly equal to 3.0 (world units).
volume_extent_world = 4.0

# 1) Instantiate the raysampler.
# Here, NDCMultinomialRaysampler generates a rectangular image
# grid of rays whose coordinates follow the PyTorch3D
# coordinate conventions.
# Since we use a volume of size 128^3, we sample n_pts_per_ray=150,
# which roughly corresponds to a one ray-point per voxel.
# We further set the min_depth=0.1 since there is no surface within
# 0.1 units of any camera plane.
raysampler = NDCMultinomialRaysampler(
    image_width=render_size,
    image_height=render_size,
    n_pts_per_ray=512,
    min_depth=0.001,
    max_depth=volume_extent_world,
)


# 2) Instantiate the raymarcher.
# Here, we use the standard EmissionAbsorptionRaymarcher 
# which marches along each ray in order to render
# each ray into a single 3D color vector 
# and an opacity scalar.
raymarcher = EmissionAbsorptionRaymarcher()

# Finally, instantiate the volumetric render
# with the raysampler and raymarcher objects.
renderer = VolumeRenderer(
    raysampler=raysampler, 
    raymarcher=raymarcher,
)

# Instantiate the volumetric model.
# We use a cubical volume with the size of 
# one side = 128. The size of each voxel of the volume 
# is set to volume_extent_world / volume_size s.t. the
# volume represents the space enclosed in a 3D bounding box
# centered at (0, 0, 0) with the size of each side equal to 3.
volume_size = hparams.shape
volume_model = VolumeModel(
    renderer,
    volume_size = [volume_size] * 3, 
    voxel_size = volume_extent_world / volume_size,
).to(device)


debug_data = first(datamodule.train_dataloader())
image3d = debug_data['image3d'].to(device)
volumes = Volumes(
    features = torch.cat([image3d]*3, dim=1),
    densities = torch.ones_like(image3d) / 1000., 
    voxel_size = volume_extent_world / volume_size,
)

DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.


In [3]:
cameras = NeuralCameras(
    batch_size=hparams.batch_size,
    random = True,
)

In [4]:
# cameras = RandomCameras()

In [5]:
# print(cameras.get_camera_center()[0]) # xyz world coordinates
# print(cameras.get_projection_transform().get_matrix()[0]) # projection matrix
# print(cameras.get_world_to_view_transform().get_matrix()[0]) # world to view transform
# print(cameras.get_full_projection_transform().get_matrix()[0]) # world to view transform

In [6]:
for idx in range(hparams.batch_size):
    print(cameras.R[idx]) # rotation matrix
    print(cameras.T[idx]) # translation vector
    print(cameras.fov[idx]) # field of view
    print(cameras.aspect_ratio[idx]) # aspect ratio

tensor([[-1.0000e+00,  2.7380e-06, -2.1298e-03],
        [ 0.0000e+00,  1.0000e+00,  1.2856e-03],
        [ 2.1298e-03,  1.2856e-03, -1.0000e+00]])
tensor([4.6566e-10, 4.6566e-10, 3.2272e+00])
tensor([[42.5664]])
tensor([[1.2271]])
tensor([[-1.0000e+00,  2.7380e-06, -2.1298e-03],
        [ 0.0000e+00,  1.0000e+00,  1.2856e-03],
        [ 2.1298e-03,  1.2856e-03, -1.0000e+00]])
tensor([4.6566e-10, 4.6566e-10, 3.2272e+00])
tensor([[42.5664]])
tensor([[1.2271]])
tensor([[-1.0000e+00,  2.7380e-06, -2.1298e-03],
        [ 0.0000e+00,  1.0000e+00,  1.2856e-03],
        [ 2.1298e-03,  1.2856e-03, -1.0000e+00]])
tensor([4.6566e-10, 4.6566e-10, 3.2272e+00])
tensor([[42.5664]])
tensor([[1.2271]])
tensor([[-1.0000e+00,  2.7380e-06, -2.1298e-03],
        [ 0.0000e+00,  1.0000e+00,  1.2856e-03],
        [ 2.1298e-03,  1.2856e-03, -1.0000e+00]])
tensor([4.6566e-10, 4.6566e-10, 3.2272e+00])
tensor([[42.5664]])
tensor([[1.2271]])
